In [1]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import weakref

In [2]:
low_memory=False
df = pd.read_csv('MyFoodData-Nutrition-Facts-SpreadSheet-Release-1-4.csv')
#df.name = df.name.str.replace(' ', '_')
df.name = df.name.apply(lambda x: x[:18])#shortens item names to 18 characters
#df = df.name[df.name=='Beef_Variety_Meats'])
data = df.sort_values(by=['name'])

#selects the columns for the graph plot (first 25 excluding water)
df2 = df[df.columns[1:26]]
df3 = df2[df2.columns[0:25]]

#calculating the %DV for each nutrient
df3['Calories'] = df2['Calories']/2000 * 100
df3['Fat (g)'] = df2['Fat (g)']/78 * 100
df3['Protein (g)'] = df2['Protein (g)']/50 * 100
df3['Carbohydrate (g)'] = df2['Carbohydrate (g)']/300 * 100 
df3['Sugars (g)'] = df2['Sugars (g)']/50 * 100
df3['Fiber (g)'] = df2['Fiber (g)']/28 * 100
df3['Cholesterol (mg)'] = df2['Cholesterol (mg)']/300 * 100
df3['Saturated Fats (g)'] = df2['Saturated Fats (g)']/20 * 100
df3['Calcium (mg)'] = df2['Calcium (mg)']/1300 * 100
df3['Iron, Fe (mg)'] = df2['Iron, Fe (mg)']/18 * 100
df3['Potassium, K (mg)'] = df2['Potassium, K (mg)']/4700 * 100
df3['Magnesium (mg)'] = df2['Magnesium (mg)']/420 * 100
df3['Vitamin A, IU (IU)'] = df2['Vitamin A, IU (IU)']/3000 * 100
df3['Vitamin A, RAE (mcg)'] = df2['Vitamin A, RAE (mcg)']/900 * 100
df3['Vitamin C (mg)'] = df2['Vitamin C (mg)']/90 * 100
df3['Vitamin B-12 (mcg)'] = df2['Vitamin B-12 (mcg)']/2.4 * 100
df3['Vitamin D (mcg)'] = df2['Vitamin D (mcg)']/20 * 100
df3['Vitamin E (Alpha-Tocopherol) (mg)'] = df2['Vitamin E (Alpha-Tocopherol) (mg)']/15 * 100
df3.rename(columns = {'Vitamin E (Alpha-Tocopherol) (mg)':'Vitamin E (mg)'}, inplace = True)
df3['Added Sugar (g)'] = df2['Added Sugar (g)']/50 * 100
#df3['Net-Carbs (g)'] = df2['Net-Carbs (g)']/ * 100
#df3['Water (g)'] = df2['Water (g)']/ * 100
df3['Omega 3s (mg)'] = df2['Omega 3s (mg)']/1600 * 100 #%AdequateIntake
df3['Omega 6s (mg)'] = df2['Omega 6s (mg)']/1200 * 100 #%AI for women

In [3]:
# -*- encoding: utf-8 -*-


#Class Definitions and Builder

from kivy.config import Config
Config.set('graphics', 'fullscreen', '0')
from kivy.core.window import Window
from kivy.garden.matplotlib.backend_kivyagg import FigureCanvasKivyAgg
from kivy.properties import ListProperty, StringProperty, NumericProperty, ObjectProperty, BooleanProperty
from kivy.uix.widget import Widget
from kivy.uix.button import Button
from kivy.uix.label import Label
from kivy.uix.dropdown import DropDown
from kivy.uix.textinput import TextInput
from kivy.lang import Builder
import pandas as pd
import numpy as np

Window.size=(700,700)
Window.clearcolor = (.95, .95, .95, 1)

Builder.load_string('''
<Button>
    size_hint:(.3, None)
    #width:100
    height:30

#<Chooser@TextInput>:
    #id: food
    #hint_text:'Enter a Food Item'
    #size_hint:(0.3,None)
    #pos_hint:{'x':0.6, 'top':.97}
    #height:30
    #width:100
    #background_normal: ''
    #background_color: (1,1,1,1)
    
<Label>
    size_hint:(0.3,None)
    height:30
    
<MyLayout>
    top_layout: topLayoutID
    graph:graph
    food_textinput:food_textinput
    quantity:quantity
    display:display
    
    graph_controls:graph_controls
    #error_label:error_label
    #foodbtn: remove
    
    FloatLayout:
        id: topLayoutID
        size: root.width, root.height
        
        Label:
            text: "Food:"
            background_color:(0, 0, 0, 1)
            pos_hint: {'x':0, 'top':.955}
            canvas.before:
                Color:
                    rgba: self.background_color
                Rectangle:
                    size: self.size
                    pos: self.pos
        Chooser:
            id: food_textinput
            hint_text:'Enter a Food Item'
            size_hint:(0.3,None)
            pos_hint:{'x':0.3, 'top':.955}
            height:30
            width:100
            #background_normal: ''
            #background_color: (1,1,1,1)
            
        TextInput:
            id:quantity
            hint_text:'#?'
            size_hint:(.1, None)
            pos_hint:{'x':0.6, 'top':.955}
            height:30
                
        Button:
            text:'Confirm'
            color: 1,1,1,1
            background_normal:''
            background_color:(220/255, 20/255, 60/255, 1)
            pos_hint:{'x':0.70, 'top':.955}
            on_press:root.run_functions()
            
        GridLayout:
            id: graph
            cols:1
            pos_hint:{'left':1,'top':0.9}
            size_hint:(0.5,0.9)
            canvas.before:
                Color:
                    rgba: (122/255, 122/255, 122/255,1)
                Rectangle:
                    size: self.size
                    pos: self.pos
        FloatLayout:
            Button:
                id:display
                color: (0,0,0,1)
                markup:True
                pos_hint:{'left':1,'top':.9}
                text:'Display value: [b][color=#ff2200]OFF[/color][/b]'
                size_hint:None,None
                width: self.texture_size[0] + dp(10)
                height: self.texture_size[1] + dp(10) 
                background_normal:''
                background_color: (1,1,1,1)
                #halign:'center'
                on_release:root.display_on()
                
        GridLayout:
            id: graph_controls
            pos_hint:{'right':1,'top':0.9}
            cols:5
            rows:5
            padding:20
            spacing:20
            size_hint:(0.5,0.9)
            #size: root.width/2, root.height
            canvas.before:
                Color:
                    rgba: (54/255, 54/255, 54/255,1)
                Rectangle:
                    size: self.size
                    pos: self.pos
    ''')
class NumLabel(Label):
    def __init__(self, quantitynum, **kwargs):
        self.quantitynum = quantitynum
        super(NumLabel, self).__init__(**kwargs)
class ButtonGraph(Button):
    def __init__(self, dynamic_id, **kwargs):
        self.dynamic_id = dynamic_id
        super(ButtonGraph, self).__init__(**kwargs)
        #self.height=30
class MyLayout(Widget):
    choices = data.name
   
    remove_called = BooleanProperty(False)
    display_value_on = BooleanProperty(False)
    delete_col = StringProperty('')
    #foodbtn = ObjectProperty(None)
    i = NumericProperty(0)
    count = NumericProperty(0)
    button_group = NumericProperty(0)
    quantitynum=NumericProperty()
    num = NumericProperty()
    first_plot = NumericProperty(0)
    userinput = ListProperty([])
    quantityinput = ListProperty([])
    xtext = StringProperty('')
    ytext = StringProperty('')
    ztext = StringProperty('')
    
    def display_on(self):
        if self.display_value_on == True:
            self.display_value_on=False
            self.ids.display.text = 'Display value: [b][color=#ff2200]OFF[/color][/b]'
            #self.ids.display.background_color = (1,0,0,1)
        else:
            self.display_value_on=True
            self.ids.display.text = 'Display value: [b][color=#5ac926]ON[/color][/b]'
            #self.ids.display.background_color = (0,1,0,1)
        self.graph_item()
        self.display_graph()
        
    def clear(self):
        self.ids.food_textinput.text = ''
        self.ids.quantity.text = ''
            
    def graph_item(self):
        self.count=0
        self.xtext = self.ytext = self.ztext = ''
        print('len(self.userinput): ', len(self.userinput))
        print('before append --> self.userinput: ', self.userinput)
        print('before append --> self.quantityinput ', self.quantityinput)
        i = 0
        loop = 0
        if self.ids.food_textinput.text != '':
            self.userinput.append(self.ids.food_textinput.text)
        for i in range(len(self.userinput)):
            if self.userinput[i] != '':
                print ('Loop: ', loop)
                print ('i: ', i)
                if self.count==0:
                    self.xtext = f'(data.index==\"{self.userinput[i]}\")'
                    self.ztext = self.xtext
                    print('self.ztext: ', self.ztext)
                else:
                    self.ytext = f'|(data.index==\"{self.userinput[i]}\")'
                    self.ztext = self.xtext + self.ytext
                    self.xtext = self.ztext
                    print('self.ztext: ', self.ztext)
                loop += 1
                self.count += 1
       # print('self.ztext: ', self.ztext)          
        if self.ids.quantity.text != '':
            self.quantityinput.append(self.ids.quantity.text)
        print('len(self.userinput): ', len(self.userinput))
        print('after append --> self.userinput: ', self.userinput)
        print('after append --> self.quantityinput ', self.quantityinput)
        print('self.count: ', self.count)
        

        #matplot graphing
        if self.remove_called == True:
            data = pd.read_pickle('data.pkl')
            print('matplot from pickle\n:', data)
            self.remove_called=False
        else:
            mpl.rc('xtick', labelsize=10)

            data = df3.round(2)
            data = data.sort_values(by=['name'])
            data = data.set_index('name')
            
            data = data[eval(self.ztext)]#|(data.name=='Bananas')]
            group1 = list(data.columns[1:21])
            group2 = list(data.columns[22:24])
            new_cols = group1+group2
            set(data.columns)==set(new_cols)
            data = data[new_cols]

            data = data.T
            data.to_pickle('data.pkl')
        
            
        #print(data.Apples)
        #print(data)
        colors = ["#f94144","#f3722c","#f8961e","#f9c74f","#90be6d","#43aa8b","#577590"]
        
        
        l = self.userinput[:len(self.userinput)] #convert userinputs into list
        l[:] = [x for x in l if x != ''] #gets rid of '' from userinput list
        print('l :',l)
        l.sort() #sort in ascending order
        convertL = ' '.join([str(e) for e in l]) #join elements of l into a string with spaces between each element
        new_convertL = convertL.replace(" ", ", ")
        
        data2 = data
        if self.ids.quantity.text != '':
            self.num = int(self.ids.quantity.text)
            
        #if self.num > 1: #if quantity of item is more than 1
        for i in range(len(self.userinput)):
            if self.userinput[i] !='':
                #print('fooditem: ', fooditem)
                data2[self.userinput[i]] *= int(self.quantityinput[i]) #multiple item data by quantity 
                #data.index(fooditem)
            
        if self.first_plot > 0: #if this is not the first graph plot
            plt.cla() #clear graph plot info
            self.graph.clear_widgets() #clear widget
        try:
        #if self.userinput:
        #fig = plt.figure()
        #if fig.get_axes():
            ax = data2.plot(kind='bar', title=f'Total %DV for {new_convertL}', width=0.8, color=colors, fontsize=12, legend=True, stacked=True)

            #graph labels
            for rect in ax.patches:
                # Find where everything is located
                height = rect.get_height()
                width = rect.get_width()
                x = rect.get_x()
                y = rect.get_y()

                # The height of the bar is the data value and can be used as the label
                label_text = f'{height:.2f}'  # f'{height:.2f}' to format decimal values

                # ax.text(x, y, text)
                label_x = x + width / 2
                label_y = y + height / 2

                # plot only when height is greater than specified value
                if height >= 1.5:
                    display_value=True
                    if label_y > 115 and y<115:
                        if height < 115:
                            label_y = 115
                        else:
                            label_y = y + 10
                    elif label_y > 115 and y>115:
                        display_value=False
                    elif height > 115 and y<115:
                        label_y = 115
                    if display_value == True:
                        if self.display_value_on == True:
                            ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=8, fontweight='bold')
                 

            plt.legend(l)
            plt.ylabel('%DV of Total Recommended', fontsize=15, fontweight='bold')
            plt.xlabel('Nutrients', fontsize=15, fontweight='bold')
            ax.set_ylim(0,120)
            plt.axhline(100, color="gray", lw=2, ls='--')
            #plt.tight_layout()
            plt.gcf().subplots_adjust(left=0.25,bottom=0.35)
            print('plot from try:\n', data)
        except:
        #else:
            self.display_no_data()
            
    def display_no_data(self):
        plt.cla() #clear graph plot info
        self.graph.clear_widgets() #clear widget
        error_label=Label(text='No Data to Display')
        self.graph.add_widget(error_label)
        self.userinput = []
        self.quantityinput = []
        #self.xtext = self.ytext = self.ztext = ''
        #self.count=0
        #self.remove_called = True
        print('self.remove_called: ', self.remove_called)
        #self.ids.error_label.text = 'No Data to Display'
        #print('plot from except:\n', data)
        
    def display_graph(self):
        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        fig.savefig('testgraph.png', dpi=100)
        self.graph.add_widget(FigureCanvasKivyAgg(fig))        
        self.first_plot +=1
        
    def add_button(self):
        #for item in self.userinput:
        #if len(self.userinput)-1 >= 0:
        length = len(self.userinput)-1
        #else:
        #    length=0
        print('before btns are made: self.userinput[-1] = ', self.userinput[length])
        foodbtn = Label(
                    text=self.userinput[length], #text = userinput at last position 
                    size_hint=(.3, None), 
                    height=30)

        numbtn = NumLabel(
                    quantitynum=self.quantityinput[length],
                    #text=f'x {numbtn.quantitynum}', 
                    size_hint=(.1, None), 
                    height=30,
                    bold=True)

        minusbtn = ButtonGraph(
                    dynamic_id=length,
                    text='-', 
                    size_hint=(.1, None), 
                    height=30, 
                    color= (0,0,0,1), 
                    background_normal= '',
                    background_color= (1,1,1,1),
                    bold=True,
                    on_release=self.reduce)#(self.ids.food_textinput.text))
        
        plusbtn = ButtonGraph(
                    dynamic_id=length,
                    text='+', 
                    size_hint=(.1, None), 
                    height=30, 
                    color= (0,0,0,1), 
                    background_normal= '',
                    background_color= (1,1,1,1),
                    bold=True,
                    on_release=self.add)#(self.ids.food_textinput.text))
        
        removebtn = ButtonGraph(
                    dynamic_id=length,
                    text='Remove', 
                    size_hint=(.3, None), 
                    height=30, 
                    color= (0,0,0,1), 
                    background_normal= '',
                    background_color= (1,1,1,1),
                    bold=True,
                    on_release=self.remove)#(self.ids.food_textinput.text))
        
        numbtn.text = f'x {numbtn.quantitynum}'

        self.graph_controls.add_widget(foodbtn)
        self.graph_controls.add_widget(numbtn)
        self.graph_controls.add_widget(minusbtn)
        self.graph_controls.add_widget(plusbtn)
        self.graph_controls.add_widget(removebtn)

        self.ids['foodbtn'+ str(length)] = foodbtn
        self.ids['numbtn'+ str(length)] = numbtn
        self.ids['minusbtn'+ str(length)] = minusbtn
        self.ids['plusbtn'+ str(length)] = plusbtn
        self.ids['removebtn' + str(length)]= removebtn
        
        
        '''print("self.ids={}".format(self.ids))
        print("self.ids[self.button_group]={}".format(self.ids[self.button_group]))
        print("self.ids[self.button_group].text={}".format(self.ids[self.button_group].text))
        #print("self.ids.test.text={}".format(self.ids.test.text))
        for key, val in self.ids.items():
            print("key={0}, val={1}".format(key, val))'''
        
        #self.button_group += 1'''
        
    def add(self, button):
        id_of_button_pressed = button.dynamic_id
        print(id_of_button_pressed)
        self.remove_called = True
        data = pd.read_pickle('data.pkl')
        #print (data)
        
        #try:
            #delete_col = StringProperty('')
            
        if int(self.quantityinput[button.dynamic_id])>0: #if quantityinput for numbtn is more than 1
            self.quantityinput[button.dynamic_id] = int(self.quantityinput[button.dynamic_id])+1 #subtract 1 from the element in that position index of quantityinput list
            self.ids['numbtn' + str(button.dynamic_id)].quantitynum = int(self.ids['numbtn' + str(button.dynamic_id)].quantitynum) + 1 #subtract 1 from quantitynum
            
            newquantitynum = self.ids['numbtn' + str(button.dynamic_id)].quantitynum
            #self.num = newquantitynum
            
            self.ids['numbtn' + str(button.dynamic_id)].text = f'x {newquantitynum}'
            
            print('self.ids[numbtn + str(button.dynamic_id)].quantitynum: ', self.ids['numbtn' + str(button.dynamic_id)].quantitynum)
            
        self.graph_item()
        self.display_graph()
            
    def reduce(self, button):#, food_item):
        id_of_button_pressed = button.dynamic_id
        print(id_of_button_pressed)
        self.remove_called = True
        data = pd.read_pickle('data.pkl')
        #print (data)
        
        #try:
            #delete_col = StringProperty('')
            
        if int(self.quantityinput[button.dynamic_id])>1: #if quantityinput for numbtn is more than 1
            self.quantityinput[button.dynamic_id] = int(self.quantityinput[button.dynamic_id])-1 #subtract 1 from the element in that position index of quantityinput list
            self.ids['numbtn' + str(button.dynamic_id)].quantitynum = int(self.ids['numbtn' + str(button.dynamic_id)].quantitynum) - 1 #subtract 1 from quantitynum
            
            newquantitynum = self.ids['numbtn' + str(button.dynamic_id)].quantitynum
            #self.num = newquantitynum
            
            self.ids['numbtn' + str(button.dynamic_id)].text = f'x {newquantitynum}'
            
            print('self.ids[numbtn + str(button.dynamic_id)].quantitynum: ', self.ids['numbtn' + str(button.dynamic_id)].quantitynum)
            
        '''else:
            delete_col = self.userinput[button.dynamic_id] #remove column of name at that position in self.userinput from dataframe
            #print(self.userinput[button.dynamic_id])
            data = data.drop(delete_col, axis=1)
            data.to_pickle('data.pkl')
            
            self.userinput[button.dynamic_id]='' #remove item from userinput list
            self.quantityinput[button.dynamic_id]='' #remove the quantity of the item from quantityinput list
            
        
            self.graph_controls.remove_widget(self.ids['foodbtn' + str(button.dynamic_id)])
            self.graph_controls.remove_widget(self.ids['numbtn' + str(button.dynamic_id)])
            self.graph_controls.remove_widget(self.ids['minusbtn' + str(button.dynamic_id)])
            self.graph_controls.remove_widget(self.ids['plusbtn' + str(button.dynamic_id)])
            print('from remove:\n', data)
            #self.count=0'''
            
        if not self.userinput:
            self.display_no_data()
        self.graph_item()
        self.display_graph()
        #self.count=0
        #print('done')
        #except:
        #    pass
        #self.graph_controls.remove_widget(self.ids.remove)
        #self.graph_controls.remove_widget(self.ids.remove2)
    def remove(self, button):#, food_item):
        id_of_button_pressed = button.dynamic_id
        print(id_of_button_pressed)
        self.remove_called = True
        data = pd.read_pickle('data.pkl')
        #print (data)
        
        #try:
            #delete_col = StringProperty('')
            
        #if int(self.quantityinput[button.dynamic_id])==1: #if quantityinput for numbtn is more than 1
        delete_col = self.userinput[button.dynamic_id] #remove column of name at that position in self.userinput from dataframe
        #print(self.userinput[button.dynamic_id])
        data = data.drop(delete_col, axis=1)
        data.to_pickle('data.pkl')

        self.userinput[button.dynamic_id]='' #remove item from userinput list
        self.quantityinput[button.dynamic_id]='' #remove the quantity of the item from quantityinput list


        self.graph_controls.remove_widget(self.ids['foodbtn' + str(button.dynamic_id)])
        self.graph_controls.remove_widget(self.ids['numbtn' + str(button.dynamic_id)])
        self.graph_controls.remove_widget(self.ids['minusbtn' + str(button.dynamic_id)])
        self.graph_controls.remove_widget(self.ids['plusbtn' + str(button.dynamic_id)])
        self.graph_controls.remove_widget(self.ids['removebtn' + str(button.dynamic_id)])
        print('from remove:\n', data)
        #self.count=0
            
        if not self.userinput:
            self.display_no_data()
        self.graph_item()
        self.display_graph()
        #self.count=0
        #print('done')
        #except:
        #    pass
        #self.graph_controls.remove_widget(self.ids.remove)
        #self.graph_controls.remove_widget(self.ids.remove2)    
        
        
    def run_functions(self):
        if self.ids.food_textinput.text == '' or self.ids.quantity.text.isdigit()==False:
            pass
        
        elif self.ids.food_textinput.text.startswith(' '):
            self.clear()
            
        elif self.ids.food_textinput.text not in self.ids.food_textinput.choiceslist:
            self.ids.food_textinput.hint_text='Item Not Found. Please Try Again'
            self.clear()
            
        elif self.ids.food_textinput.text in self.userinput:
            self.ids.food_textinput.hint_text='Item has already been entered'
            self.clear()
            
        else:
            self.ids.food_textinput.hint_text='Enter a Food Item'
            self.graph_item()
            self.display_graph()
            self.add_button()
            self.clear()
            #if self.first_plot>0 and self.userinput:
                #self.i +=1
            
    
    
class Chooser(TextInput):
    choiceslist = ListProperty([])

    def __init__(self, **kwargs):
        choices = data.name
        
        self.choiceslist = choices  # list of choices
        super(Chooser, self).__init__(**kwargs)
        self.multiline = False
        self.halign = 'left'
        self.bind(text=self.on_text)
        self.dropdown = None
        
    def open_dropdown(self, *args):
        if self.dropdown:
            self.dropdown.open(self)

    def keyboard_on_key_down(self, window, keycode, text, modifiers):
        if self.suggestion_text and keycode[0] == ord('\r'):  # enter selects current suggestion
            self.suggestion_text = ' '  # setting suggestion_text to '' screws everything
            self.text = self.values[0]
                        
            if self.dropdown:
                self.dropdown.dismiss()
                self.dropdown = None
        else:
            super(Chooser, self).keyboard_on_key_down(window, keycode, text, modifiers)

    def on_text(self, chooser, text):
        if self.dropdown:
            self.dropdown.dismiss()
            self.dropdown = None
        if text == '':
            return
        values = []
        
        for addr in self.choiceslist:#for each item in choicelist, check if text matches if text>2
            newaddr = addr.lower()#convert to lowercase
            if len(self.text)> 2:
                if newaddr.startswith(text.lower()) and addr not in values:
                    values.append(addr)
                    values = sorted(values, key=len)#sort by length, ascending order
                    values = values[:5] 
                    
        for addr in self.choiceslist:
            if len(self.text)> 2:
                newaddr = addr.lower()
                if text.lower() in newaddr and addr not in values and len(values) < 5:
                        values.append(addr)
                        values = sorted(values, key=len)
                        values = values[:5]
                        
        self.values = values
        if len(values) > 0:
            if len(self.text) < len(self.values[0]):
                self.suggestion_text = self.values[0][len(self.text):]
            else:
                self.suggestion_text = ' '  # setting suggestion_text to '' screws everything
            self.dropdown = DropDown()
            for val in self.values:
                btn = Button(
                    text=val, 
                    size_hint_y=None, 
                    height=30, 
                    color= (0,0,0,1), 
                    background_normal= '',
                    background_color= (1,1,1,1),
                    on_release=self.do_choose)
                
                self.dropdown.add_widget(btn)
            self.dropdown.open(self)

    def do_choose(self, butt):
        self.text = butt.text
        if self.dropdown:
            self.dropdown.dismiss()
            self.dropdown = None
    
if __name__ == '__main__':
    from kivy.app import App
    from kivy.uix.relativelayout import RelativeLayout

    class TestApp(App):
        top_layout = ObjectProperty(None)
        def build(self):
            return MyLayout()


    TestApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\k_a_j\.kivy\logs\kivy_21-05-15_6.txt
[INFO   ] [Kivy        ] v2.0.0
[INFO   ] [Kivy        ] Installed at "D:\Programs\Programs\Anaconda\envs\foodapp-test\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 15:50:08) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "D:\Programs\Programs\Anaconda\envs\foodapp-test\python.exe"
[INFO   ] [Factory     ] 186 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.6.0 - Build 27.20.100.8729'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) UHD Graphics'>
[INFO

len(self.userinput):  0
before append --> self.userinput:  []
before append --> self.quantityinput  []
Loop:  0
i:  0
self.ztext:  (data.index=="Apples")
len(self.userinput):  1
after append --> self.userinput:  ['Apples']
after append --> self.quantityinput  ['2']
self.count:  1
l : ['Apples']
plot from try:
 name                  Apples
Calories                5.20
Fat (g)                 0.44
Protein (g)             1.04
Carbohydrate (g)        9.20
Sugars (g)             41.56
Fiber (g)              17.14
Cholesterol (mg)        0.00
Saturated Fats (g)      0.28
Calcium (mg)            0.92
Iron, Fe (mg)           1.34
Potassium, K (mg)       4.56
Magnesium (mg)          2.38
Vitamin A, IU (IU)      3.60
Vitamin A, RAE (mcg)    0.66
Vitamin C (mg)         10.22
Vitamin B-12 (mcg)      0.00
Vitamin D (mcg)         0.00
Vitamin E (mg)          2.40
Added Sugar (g)          NaN
Net-Carbs (g)          22.82
Omega 3s (mg)           1.12
Omega 6s (mg)           7.16
before btns are made:

0
self.ids[numbtn + str(button.dynamic_id)].quantitynum:  4
len(self.userinput):  1
before append --> self.userinput:  ['Apples']
before append --> self.quantityinput  [4]
Loop:  0
i:  0
self.ztext:  (data.index=="Apples")
len(self.userinput):  1
after append --> self.userinput:  ['Apples']
after append --> self.quantityinput  [4]
self.count:  1
matplot from pickle
: name                  Apples
Calories                2.60
Fat (g)                 0.22
Protein (g)             0.52
Carbohydrate (g)        4.60
Sugars (g)             20.78
Fiber (g)               8.57
Cholesterol (mg)        0.00
Saturated Fats (g)      0.14
Calcium (mg)            0.46
Iron, Fe (mg)           0.67
Potassium, K (mg)       2.28
Magnesium (mg)          1.19
Vitamin A, IU (IU)      1.80
Vitamin A, RAE (mcg)    0.33
Vitamin C (mg)          5.11
Vitamin B-12 (mcg)      0.00
Vitamin D (mcg)         0.00
Vitamin E (mg)          1.20
Added Sugar (g)          NaN
Net-Carbs (g)          11.41
Omega 3s (mg)        

0
self.ids[numbtn + str(button.dynamic_id)].quantitynum:  4
len(self.userinput):  2
before append --> self.userinput:  ['Apples', 'Oranges']
before append --> self.quantityinput  [4, '2']
Loop:  0
i:  0
self.ztext:  (data.index=="Apples")
Loop:  1
i:  1
self.ztext:  (data.index=="Apples")|(data.index=="Oranges")
len(self.userinput):  2
after append --> self.userinput:  ['Apples', 'Oranges']
after append --> self.quantityinput  [4, '2']
self.count:  2
matplot from pickle
: name                  Apples  Oranges
Calories                2.60     2.35
Fat (g)                 0.22     0.15
Protein (g)             0.52     1.88
Carbohydrate (g)        4.60     3.92
Sugars (g)             20.78    18.70
Fiber (g)               8.57     8.57
Cholesterol (mg)        0.00     0.00
Saturated Fats (g)      0.14     0.08
Calcium (mg)            0.46     3.08
Iron, Fe (mg)           0.67     0.56
Potassium, K (mg)       2.28     3.85
Magnesium (mg)          1.19     2.38
Vitamin A, IU (IU)      1.80 

1
self.ids[numbtn + str(button.dynamic_id)].quantitynum:  1
len(self.userinput):  2
before append --> self.userinput:  ['Apples', 'Oranges']
before append --> self.quantityinput  [1, 1]
Loop:  0
i:  0
self.ztext:  (data.index=="Apples")
Loop:  1
i:  1
self.ztext:  (data.index=="Apples")|(data.index=="Oranges")
len(self.userinput):  2
after append --> self.userinput:  ['Apples', 'Oranges']
after append --> self.quantityinput  [1, 1]
self.count:  2
matplot from pickle
: name                  Apples  Oranges
Calories                2.60     2.35
Fat (g)                 0.22     0.15
Protein (g)             0.52     1.88
Carbohydrate (g)        4.60     3.92
Sugars (g)             20.78    18.70
Fiber (g)               8.57     8.57
Cholesterol (mg)        0.00     0.00
Saturated Fats (g)      0.14     0.08
Calcium (mg)            0.46     3.08
Iron, Fe (mg)           0.67     0.56
Potassium, K (mg)       2.28     3.85
Magnesium (mg)          1.19     2.38
Vitamin A, IU (IU)      1.80     

...

<Figure size 350x630 with 1 Axes>

Exception: You must specify a format to save into a BytesIO object

<Figure size 350x630 with 1 Axes>

Exception: You must specify a format to save into a BytesIO object

<Figure size 350x630 with 1 Axes>